# Electricity Prices – Daily Feature Pipeline

Fetch yesterday's electricity prices and weather for Stockholm and upsert them into the existing Hopsworks feature groups so they stay aligned with the backfill schema (unix_time PK etc.).

In [2]:
from pathlib import Path
import sys
import datetime
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

from dotenv import load_dotenv
import hopsworks

# 1. Find project root (one level up from notebooks/)
root_dir = Path("..").resolve()

# 2. Add project root to PYTHONPATH so we can import the src package
if str(root_dir) not in sys.path:
    sys.path.append(str(root_dir))

# 3. Load .env from project root
env_path = root_dir / ".env"
load_dotenv(env_path)

# 4. Load settings and utility functions (after adjusting PYTHONPATH)
from src.config import ElectricitySettings
from src import util

settings = ElectricitySettings()

# 5. Log in to Hopsworks and get feature store
project = hopsworks.login()
fs = project.get_feature_store(name='scalableproject_featurestore')


print("Successfully logged in to Hopsworks project:", settings.HOPSWORKS_PROJECT)


ElectricitySettings initialized
2025-12-11 11:51:40,926 INFO: Initializing external client
2025-12-11 11:51:40,927 INFO: Base URL: https://eu-west.cloud.hopsworks.ai:443
2025-12-11 11:51:42,222 INFO: Python Engine initialized.

Logged in to project, explore it here https://eu-west.cloud.hopsworks.ai:443/p/127
Successfully logged in to Hopsworks project: ScalableProject


In [ ]:
# Get the feature groups
electricity_prices_fg = fs.get_feature_group('electricity_prices', version=1)
weather_hourly_fg = fs.get_feature_group('weather_hourly', version=1)


In [4]:
# Configuration
PRICE_AREA = "SE3"  # Stockholm / South-Central Sweden
CITY = "Stockholm"
LATITUDE = 59.3251   # Stockholm coordinates
LONGITUDE = 18.0711

#LATITUDE, LONGITUDE = util.get_city_coordinates(CITY)

today = datetime.date.today()
yesterday = today - datetime.timedelta(days=1)

## Step 1 — Fetch and upsert yesterday's electricity prices
Pull yesterday's hourly prices, align them to the backfill schema (including `unix_time` as PK) and write to the `electricity_prices` feature group.

In [5]:
# Fetch yesterday's actual electricity prices
df_prices = util.fetch_electricity_prices(
    start_date=yesterday,
    end_date=yesterday,
    price_area=PRICE_AREA,
    show_progress=False,
    request_pause=0,
)
df_prices = util.align_electricity_price_schema(df_prices)
# Use same label as backfill (CITY) for PK consistency
df_prices["price_area"] = CITY
# Add unix_time for primary key (ms since epoch UTC)
df_prices["timestamp"] = pd.to_datetime(df_prices["timestamp"], utc=True)
df_prices["unix_time"] = df_prices["timestamp"].astype("int64") // 10**6
price_columns = ['unix_time', 'timestamp', 'date', 'hour', 'price_area', 'price_sek']
df_prices = df_prices[price_columns]
print(f"Fetched {len(df_prices)} rows for {yesterday}")

# Insert new data
electricity_prices_fg.insert(df_prices, storage="online", wait=True)

Fetching electricity prices from 2025-12-10 to 2025-12-10 for SE3...
Fetched 24 hourly price records across 1 day(s)
2025-12-11 11:51:44,595 WARNING: DeprecationWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.

2025-12-11 11:51:44,595 WARNING: DeprecationWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.

Fetched 24 rows for 2025-12-10
2025-12-11 11:51:44,795 INFO: 	2 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://eu-west.cloud.hopsworks.ai:443/p/127/fs/74/fg/3120


Uploading Dataframe: 100.00% |██████████| Rows 24/24 | Elapsed Time: 00:00 | Remaining Time: 00:00


Launching job: electricity_prices_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://eu-west.cloud.hopsworks.ai:443/p/127/jobs/named/electricity_prices_1_offline_fg_materialization/executions
2025-12-11 11:51:57,087 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-12-11 11:52:00,248 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-12-11 11:53:50,931 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2025-12-11 11:53:51,065 INFO: Waiting for log aggregation to finish.
2025-12-11 11:53:59,806 INFO: Execution finished successfully.


Online data ingestion progress: 0.00% |          | Rows 0/24

(Job('electricity_prices_1_offline_fg_materialization', 'PYSPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "price_sek",
           "min_value": -5.0,
           "max_value": 50.0,
           "strict_min": false
         },
         "meta": {
           "expectationId": 3079
         }
       },
       "result": {
         "observed_value": 0.14372999966144562,
         "element_count": 24,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-12-11T10:51:44.000795Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_typ

## Step 2 — Fetch and upsert yesterday's weather
Fetch yesterday's hourly actual weather, align to the backfill schema, and write to the `weather_hourly` feature group.

In [6]:
# Fetch and upload yesterday's actual hourly weather
actual_weather_yesterday = util.get_yesterday_hourly_weather(
    latitude=LATITUDE,
    longitude=LONGITUDE,
    city=CITY,
)
actual_weather_yesterday['date'] = pd.to_datetime(actual_weather_yesterday['date'])
actual_weather_yesterday['timestamp'] = pd.to_datetime(actual_weather_yesterday['timestamp'], utc=True)
# Add unix_time for PK (ms since epoch UTC)
actual_weather_yesterday['unix_time'] = actual_weather_yesterday['timestamp'].astype('int64') // 10**6

# Keep the same column order/schema as in the backfill FG
aweather_cols = [
    "unix_time",
    "timestamp",
    "temperature_2m",
    "apparent_temperature",
    "precipitation",
    "rain",
    "snowfall",
    "cloud_cover",
    "wind_speed_10m",
    "wind_speed_100m",
    "wind_direction_10m",
    "wind_direction_100m",
    "wind_gusts_10m",
    "surface_pressure",
    "city",
    "date",
    "hour",
]
actual_weather_yesterday = actual_weather_yesterday[aweather_cols]

if len(actual_weather_yesterday):
    weather_hourly_fg.insert(actual_weather_yesterday, storage="online", wait=True)
    print(f"Inserted actual weather for yesterday: {len(actual_weather_yesterday)} rows for {yesterday}")
else:
    print("No actual weather rows for yesterday.")


Fetching historical weather for Stockholm (59.3251, 18.0711)...
Date range: 2025-12-10 to 2025-12-10
Coordinates: 59.29701232910156°N 18.163265228271484°E
Elevation: 23.0 m asl
Fetched 24 hourly weather records
2025-12-11 11:54:00,413 INFO: 	3 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://eu-west.cloud.hopsworks.ai:443/p/127/fs/74/fg/3121


Uploading Dataframe: 100.00% |██████████| Rows 23/23 | Elapsed Time: 00:00 | Remaining Time: 00:00


Launching job: weather_hourly_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://eu-west.cloud.hopsworks.ai:443/p/127/jobs/named/weather_hourly_1_offline_fg_materialization/executions
2025-12-11 11:54:12,453 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-12-11 11:54:15,612 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-12-11 11:56:15,841 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2025-12-11 11:56:15,970 INFO: Waiting for log aggregation to finish.
2025-12-11 11:56:24,640 INFO: Execution finished successfully.


Online data ingestion progress: 0.00% |          | Rows 0/23

Inserted actual weather for yesterday: 23 rows for 2025-12-10


In [ ]:
# Just a sanity check
historical_df = electricity_prices_fg.read()
historical_df = historical_df.sort_values("timestamp", ascending=True).reset_index(drop=True)
historical_df.tail(24)
ts_local = df_prices['timestamp'].dt.tz_localize('UTC').dt.tz_convert('Europe/Stockholm')
df_prices['date_local'] = ts_local.dt.date
df_prices['hour_local'] = ts_local.dt.hour

print(len(df_prices[df_prices['date_local'] == yesterday]))  # should be 24

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.68s) 
24
